# Data Investigation

In [ ]:
options(jupyter.plot_mimetypes = "image/svg+xml")
library(tidyverse)

In [ ]:
round_by <- function(x, divisor) {
    round(x / divisor) * divisor
}

In [ ]:
training_data <- read_csv('train_processed.csv')

In [ ]:
training_data

In [ ]:
training_data %>%
    group_by(Sex) %>%
    summarize(TotalSurvived = sum(Survived))

In [ ]:
training_data %>%
    mutate(Age = round_by(Age, 5)) %>%
    group_by(Age, Sex) %>%
    summarize(TotalSurvived = sum(Survived)) %>%
    ggplot(aes(x = Age, y = TotalSurvived, fill = Sex)) +
    geom_col()

In [ ]:
training_data %>%
    group_by(Embarked, FamilySize) %>%
    summarize(TotalSurvived = sum(Survived)) %>%
    ggplot(aes(x = Embarked, y = TotalSurvived, fill = FamilySize)) +
    geom_col()

In [ ]:
training_data %>%
    mutate(Age = round_by(Age, 5)) %>%
    ggplot(aes(x = Sex, y = Age)) +
    geom_violin()

In [ ]:
training_data %>%
    ggplot(aes(x = FamilySize, y = Pclass)) +
    geom_jitter()

In [ ]:
training_data %>%
    group_by(Pclass) %>%
    summarize(min = min(Fare), lower_quartile = quantile(Fare, 0.25), median = median(Fare), upper_quartile = quantile(Fare, 0.75), max = max(Fare))